In [1]:
!pip install SimpleITK


In [2]:
import numpy as np
import shutil
import os
import SimpleITK as sitk

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def resample_image(itk_image, out_spacing=[1.0, 1.0, 1.0], is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image) 

In [ ]:
cur_dir = '/content/drive/My Drive/'
dir_with_img = os.path.join(cur_dir, 'Nii_images')

dest_dir_main = os.path.join(cur_dir, 'Resized_imgs')
list_dir = os.listdir(dir_with_img)

if not os.path.isdir(dest_dir_main):
  print(f'Creating Main directory :{dest_dir_main}\n')
  os.mkdir(dest_dir_main)

for sub_dir in list_dir:
  #### Copies the annotations:
  dest_dir_ann =  os.path.join(dest_dir_main,sub_dir)
  print('Creating : ',dest_dir_ann) 
  os.mkdir(dest_dir_ann)
  dir_to_move = os.path.join(dir_with_img,sub_dir, 'annotations')
  #Checks for existing dir to skip step  
  if not os.path.isdir(os.path.join(dest_dir_ann, 'annotations')):    
    print('Copying : ', dir_to_move,' -- to -->', dest_dir_ann)  
    shutil.copytree(dir_to_move, os.path.join(dest_dir_ann, 'annotations'))
    #shutil.move(dir_to_move, dest_dir_ann)
  
  #### Resamples Image:

  dest_dir_img = os.path.join(dest_dir_ann, 'image')
  
  #Checks for existing dir to skip step
  if not os.path.isdir(dest_dir_img):    
    print('Creating : ', dest_dir_main)
    os.mkdir(dest_dir_img)  
    
    path_to_sitk_image = os.path.join(dir_with_img,sub_dir,'image','image.nii')
    print('Reading Image at:', path_to_sitk_image)
    image = sitk.ReadImage(path_to_sitk_image)
    # Calculating new spacing
    img_size = np.asarray(image.GetSize())
    img_spacing = np.asarray(image.GetSpacing())
    my_image_size = np.asarray((256,256,305))
    new_spacing = img_spacing*(img_size/my_image_size)
    print(f'Resampling from {img_size} to {my_image_size}.\nPrevious Spacing {img_spacing}\tNew Spacing {new_spacing}')
    newimage  = resample_image(image, out_spacing=new_spacing, is_label=False)

    outputImageFileName = os.path.join(dest_dir_img,'image.nii')
    writer = sitk.ImageFileWriter()
    writer.SetFileName(outputImageFileName)
    #writer.SetUseCompression(True)
    print(f'Writing to : {outputImageFileName}\n')
    writer.Execute(newimage)
    
print('Done')